# Tile masking
Masking is a common operation to use when performing analysis across tiles. Tile masking allows one to compare two tiles, a value tile and a masking tile. The masking tile can be thought of as being placed on top of the other tile, obscuring certain values. Masking cells set to NODATA are opaque, and cells that are anything else are transparent. In this way, value cells that correspond to masking cells that are NODATA become NODATA and value cells corresponding to other values are untouched.

Initial setup:

In [3]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import geotrellis.raster.{mask => _, _}
import geotrellis.raster.render._
import org.apache.spark.sql.functions._  
import astraea.spark.rasterframes.stats.{CellHistogram=>CH}

implicit val spark = SparkSession.builder().
    master("local").appName("RasterFrames").
    config("spark.ui.enabled", "false").
    getOrCreate().
    withRasterFrames

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")

val filenamePattern = "L8-B%d-Elkton-VA.tiff"
val bandNumbers = 1 to 4
val bandColNames = bandNumbers.map(b ⇒ s"band_$b").toArray

import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import geotrellis.raster.{mask=>_, _}
import geotrellis.raster.render._
import org.apache.spark.sql.functions._
import astraea.spark.rasterframes.stats.{CellHistogram=>CH}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@80dfe0
readTiff: (name: String)geotrellis.raster.io.geotiff.SinglebandGeoTiff
filenamePattern: String = L8-B%d-Elkton-VA.tiff
bandNumbers: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3, 4)
bandColNames: Array[String] = Array(band_1, band_2, band_3, band_4)


In [4]:
val joinedRF = bandNumbers.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ (b, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"band_$b") }.
  reduce(_ spatialJoin _)

joinedRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 3 more fields]


In [25]:
joinedRF.select(cellType($"band_1")).show()
joinedRF = joinedRF.withColumn()

+--------------------------+
|celltypeexpression(band_1)|
+--------------------------+
|                 uint16raw|
+--------------------------+



Because the tile has a cellType of raw, there is not a NODATA value. One possible solution is to convert the CellType to something that does support NODATA values. This transformation is wrapped into a threshold function that creates a new tile based on whether the cell values in a tile are higher than a certain number.

In [ ]:
val threshold = udf((t: Tile) => {
  t.convert(IntConstantNoDataCellType).map(x => if (x > 10500) x else NODATA)
  } )

First, we create a masking tile based on a previous one (in this case, band_1 of the original tiff). This tile is full of either NODATA or the unchanged values corresponsing to whether the value in that cell is greater than a threshold.

In [ ]:
val withMaskedTile = joinedRF.withColumn("maskTile", threshold(joinedRF("band_1"))).asRF

In [23]:
withMaskedTile.select(noDataCells($"maskTile")).show()

+---------------------+
|noDataCells(maskTile)|
+---------------------+
|                28570|
+---------------------+



## Mask 
Mask is an operation that takes two tiles and eliminates data from one cell that corresponds to a value in the other. For instance, if a cell in the masking tile contains a NODATA, the corresponding cell in the other tile would be set to NODATA. If the masking cell contained anything else, the corresponding cell would have had its value untouched.
### Inverse Mask
Inverse mask works in the opposite way. It sets the values of all cells that correspond to masking cells that are not NODATA to NODATA and doesn't touch those that have corresponding NODATA masking cells.

In [ ]:
val masked = withMaskedTile
  .withColumn("masked", mask(joinedRF("band_2"), withMaskedTile("maskTile"))).asRF
val inversemasked = withMaskedTile
  .withColumn("inversemasked", inverseMask(joinedRF("band_2"), withMaskedTile("maskTile"))).asRF

In [ ]:
val maskRaster = masked.toRaster(masked("masked"), 466, 428)
val inverseMaskRaster = inversemasked.toRaster(inversemasked("inversemasked"), 466, 428)

val brownToGreen = ColorRamp(
  RGBA(166,97,26,255),
  RGBA(223,194,125,255),
  RGBA(245,245,245,255),
  RGBA(128,205,193,255),
  RGBA(1,133,113,255)
).stops(128)

val colors = ColorMap.fromQuantileBreaks(maskRaster.tile.histogramDouble(), brownToGreen)

maskRaster.tile.color(colors).renderPng().write("mask.png")
inverseMaskRaster.tile.color(colors).renderPng().write("inverseMask.png")

Pictures go here

In [6]:
joinedRF.show()

+-----------+--------------------+--------------------+--------------------+--------------------+
|spatial_key|              band_1|              band_2|              band_3|              band_4|
+-----------+--------------------+--------------------+--------------------+--------------------+
|      [0,0]|geotrellis.raster...|geotrellis.raster...|geotrellis.raster...|geotrellis.raster...|
+-----------+--------------------+--------------------+--------------------+--------------------+

